In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
#Load the dataset
import pandas as pd

# Load your original dataset
df = pd.read_csv('/content/dataset.csv')  # Adjust the path if needed

# Combine 'LastUpdatedDate' and 'LastUpdatedTime' into a single datetime column
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                 format="%d-%m-%Y %H:%M:%S")

# Rename for clarity: assume 'SystemCodeNumber' is a unique lot ID
df.rename(columns={"SystemCodeNumber": "LotID"}, inplace=True)

# Select only relevant columns for Model 1
df_stream = df[["LotID", "Timestamp", "Occupancy", "Capacity"]].copy()

# Sort by timestamp for proper streaming
df_stream = df_stream.sort_values("Timestamp").reset_index(drop=True)

# Save to CSV for Pathway to use
df_stream.to_csv("parking_stream_model1.csv", index=False)

# Quick check
df_stream.head()


In [ ]:
#Model 1: Baseline Linear Model
!pip install pathway bokeh panel --quiet


In [ ]:
import pandas as pd
import numpy as np
import pathway as pw
import panel as pn
import bokeh.plotting

# Enable Panel extension
pn.extension()


In [ ]:
#3 Save Preprocessed Data to CSV
# Save to a streaming-friendly format
df[["LotID", "Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream_model1.csv", index=False)


In [ ]:
#4. Define Schema & Ingest Data
# Define the schema of incoming data
class ParkingSchema(pw.Schema):
    LotID: str
    Timestamp: str
    Occupancy: int
    Capacity: int

# Ingest stream
data = pw.demo.replay_csv("parking_stream_model1.csv", schema=ParkingSchema, input_rate=1000)


In [ ]:
#5. Time Processing
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [ ]:
#6. Define Price Calculation
import datetime

# Daily tumbling window pricing
delta_window_model1 = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price = 10 + 5 * (pw.this.occ_max / pw.this.cap)
    )
)


In [ ]:
#7. Define Bokeh Visualizer
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Daily Parking Price (Model 1)",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="blue")
    fig.scatter("t", "price", source=source, size=6, color="red")  # Updated line
    return fig


In [ ]:
#8. Serve Dashboard
dashboard = pn.Column(viz)
dashboard.servable()


In [ ]:
#9. Run Pathway Pipeline (Real-time Simulation)
%%capture --no-display
pw.run()


In [ ]:
import pandas as pd

# Load your original dataset (replace with correct filename if needed)
df = pd.read_csv("/content/dataset.csv")  # Adjust path if file is elsewhere

# Create the 'Timestamp' column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Rename ID as LotID for clarity
df.rename(columns={'ID': 'LotID'}, inplace=True)

# Select only required columns for Model 1
df_stream = df[['LotID', 'Timestamp', 'Occupancy', 'Capacity']]

# Save this to CSV for Pathway to stream
df_stream.to_csv("parking_stream.csv", index=False)

# Optional: View sample
print(df_stream.head())




In [ ]:
import pandas as pd
df = pd.read_csv("parking_stream.csv")
print(df.head())


In [ ]:
#Model 1 – Baseline Linear Model
import pathway as pw
import datetime
import panel as pn
import bokeh.plotting

# Step 1: Define schema
class ParkingSchema(pw.Schema):
    LotID: str
    Timestamp: str
    Occupancy: int
    Capacity: int

# Step 2: Stream the data using Pathway
data = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=1000
)

# Step 3: Convert timestamp and add day column
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Step 4: Define daily tumbling window and compute price
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.Occupancy),
        occ_min = pw.reducers.min(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

# Step 5: Bokeh plotting function
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 1: Baseline Daily Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.scatter("t", "price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")

# Step 6: Display the dashboard
pn.Column(viz).servable()

# Step 7: Run Pathway pipeline (in background)
pw.run()



In [ ]:
import pandas as pd

# Load the cleaned dataset again (assuming you've already done preprocessing)
df = pd.read_csv("/content/dataset.csv")  # replace with your dataset path if different

# Create Timestamp column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Filter and save only necessary columns
df_filtered = df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity']].dropna()
df_filtered.columns = ['LotID', 'Timestamp', 'Occupancy', 'Capacity']  # rename for clarity

# Save to CSV
df_filtered.to_csv("parking_stream.csv", index=False)

print("✅ File 'parking_stream.csv' created successfully.")
print(df_filtered.head())


In [ ]:
#fixing emptydataframe
#Check the raw date and time columns
df = pd.read_csv("/content/dataset.csv")
print(df[['LastUpdatedDate', 'LastUpdatedTime']].head(10))


In [ ]:
#Parse Date and Time Properly
# Combine LastUpdatedDate and LastUpdatedTime into one column
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'].astype(str) + ' ' + df['LastUpdatedTime'].astype(str),
    format='%d-%m-%Y %H:%M:%S',
    errors='coerce'  # Converts invalid parsing to NaT
)



In [ ]:
#2: Check the Results
print("Total rows before filtering:", len(df))
print("Valid rows after Timestamp parsing:", df['Timestamp'].notnull().sum())


In [ ]:
# inspect what columns do exist in your current
print(df.columns.tolist())


In [ ]:
#3: Filter and Export the Relevant Columns
# let’s extract the data you need for modeling (Occupancy, Capacity, Timestamp, LotID) and save it to a CSV:
# Keep only required columns
# Prepare data for streaming
df_stream = df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity']].copy()
df_stream.columns = ['LotID', 'Timestamp', 'Occupancy', 'Capacity']  # Rename for consistency

# Sort by timestamp
df_stream = df_stream.sort_values('Timestamp').reset_index(drop=True)

# Save to CSV
df_stream.to_csv("parking_stream.csv", index=False)

print("✅ File 'parking_stream.csv' created successfully.")
print(df_stream.head())


In [ ]:
#Install & Import Required Libraries
!pip install pathway bokeh panel --quiet


In [ ]:

#Install & Import Required Libraries
import pathway as pw
import pandas as pd
import datetime
import panel as pn
import bokeh.plotting


In [ ]:
#Load the CSV as a Real-Time Stream in Pathway
# Define schema
class ParkingSchema(pw.Schema):
    LotID: str
    Timestamp: str
    Occupancy: int
    Capacity: int

# Simulated stream
stream = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=1000
)


In [ ]:
#Preprocessing & Time Conversion
fmt = "%Y-%m-%d %H:%M:%S"

stream_with_time = stream.with_columns(
    t = stream.Timestamp.dt.strptime(fmt),
    day = stream.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [ ]:
#Apply Baseline Linear Pricing Model
import datetime

# Define tumbling daily window
delta_window = (
    stream_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.Occupancy),
        occ_min = pw.reducers.min(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)


In [ ]:
#Plot Daily Price Updates
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 1: Daily Dynamic Parking Price",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.scatter("t", "price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


In [ ]:
# Run the Stream
%%capture --no-display
pw.run()


MODEL 2 :-

In [ ]:
print(df.columns)


In [ ]:
#Filling Missing Values:
# Fill missing traffic values with 0 (or use .mean() instead if preferred)
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].fillna(0)


In [ ]:
#build Model 2 pricing function.
import pandas as pd
import numpy as np

# Reload or ensure df is clean
df['Occupancy'] = pd.to_numeric(df['Occupancy'], errors='coerce')
df['Capacity'] = pd.to_numeric(df['Capacity'], errors='coerce')
df['QueueLength'] = pd.to_numeric(df['QueueLength'], errors='coerce')
df['TrafficConditionNearby'] = pd.to_numeric(df['TrafficConditionNearby'], errors='coerce')
df['IsSpecialDay'] = pd.to_numeric(df['IsSpecialDay'], errors='coerce')

# Encode VehicleType
vehicle_type_weights = {
    'Car': 1.0,
    'Bike': 0.5,
    'Truck': 1.5
}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_type_weights).fillna(1.0)

# Normalize demand calculation
df['Demand'] = (
    0.3 * (df['Occupancy'] / df['Capacity']) +
    0.2 * df['QueueLength'] +
    -0.1 * df['TrafficConditionNearby'] +
    0.2 * df['IsSpecialDay'] +
    0.2 * df['VehicleWeight']
)

# Normalize demand between 0 and 1
df['NormDemand'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())

# Set pricing
base_price = 10
lambda_ = 0.8  # weight of demand on price
df['DynamicPrice'] = base_price * (1 + lambda_ * df['NormDemand'])

# Final preview
df[['Timestamp', 'Occupancy', 'Capacity', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType', 'NormDemand', 'DynamicPrice']].head()


In [ ]:
print(df[['Timestamp', 'Occupancy', 'Capacity', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType', 'NormDemand', 'DynamicPrice']].head(10))


In [ ]:
#Step-by-Step Code to Plot Time vs Dynamic Price {using matplotlib}
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Ensure Timestamp is in datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Set the Timestamp as the index (optional but helps with plotting)
df.set_index('Timestamp', inplace=True)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['DynamicPrice'], marker='o', linestyle='-', color='blue', label='Dynamic Price')

# Formatting the date on x-axis
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1))
plt.xticks(rotation=45)

plt.title('Dynamic Parking Price Over Time', fontsize=14)
plt.xlabel('Time of Day')
plt.ylabel('Price ($)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
df.columns = df.columns.str.strip()  # removes extra spaces


In [ ]:
#Step-by-Step Code to Plot Time vs Dynamic Price {using bokeh} - interactive
print(df.columns.tolist())



In [ ]:
#Create Timestamp column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')


In [ ]:
# Remove duplicate columns (keep the first occurrence)
df = df.loc[:, ~df.columns.duplicated()]


In [ ]:
print(df.columns[df.columns.duplicated()])


In [ ]:
##Step-by-Step Code to Plot Time vs Dynamic Price {using bokeh} - interactive
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column

output_notebook()

# Create ColumnDataSource from dataframe, ignoring the index
source = ColumnDataSource(df.drop(columns=['Timestamp']).reset_index())


p = figure(
    x_axis_type='datetime',
    title='Dynamic Parking Price Over Time',
    height=400,
    width=800
)

p.line(x='Timestamp', y='DynamicPrice', source=source, line_width=2, color='navy')
p.circle(x='Timestamp', y='DynamicPrice', source=source, size=6, color='red')

p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price ($)'
from bokeh.models import HoverTool

hover = HoverTool(
    tooltips=[
        ("Time", "@Timestamp{%F %T}"),
        ("Price", "@DynamicPrice{$0.00}"),
        ("Occupancy", "@Occupancy"),
        ("Capacity", "@Capacity"),
        ("Vehicle Type", "@VehicleType")
    ],
    formatters={"@Timestamp": "datetime"},
    mode='vline'
)
p.add_tools(hover)

show(column(p))

In [ ]:
df.to_csv("Model2_Pricing_Output.csv", index=False)
print("✅ CSV Exported")
